# Lab | Summarization evaluation using LangSmith
Let's revisit your capstone project 2? Well, sort of. Pick diffierent sets of data and re-run this notebook. Maybe parts of the dataset you used in your last project week. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications using LangSmith.

What did you learn? - Let's discuss that in class

## LangSmith - LangChain evaluation

In [1]:
!pip install langsmith datasets
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency

In [3]:
#from dotenv import load_dotenv, find_dotenv
import os
#_ = load_dotenv(find_dotenv())

from google.colab import userdata

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')
LANGCHAIN_API_KEY = userdata.get("LANGCHAIN_API_KEY")
HF_TOKEN = userdata.get("HF_TOKEN")

In [4]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY
os.environ['HF_TOKEN'] = HF_TOKEN

In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="langsmith_max-test"

In [6]:
#Importing Client from Langsmith
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)

### Create Dataset


In [7]:
from datasets import load_dataset
cnn_dataset = load_dataset(
    "ccdv/cnn_dailymail", version
    ="3.0.0",
    trust_remote_code=True
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
def add_prefix(example):
    return {
        **example,
        "article": f"Summarize this news:\n{example['article']}"
    }

#cnn_dataset = cnn_dataset.map(add_prefix)

In [9]:
cnn_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [10]:
cnn_dataset['train'][0]

{'article': 'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but bec

In [11]:
#Get just a few news to test
MAX_NEWS=10
sample_cnn = cnn_dataset["test"].select(range(MAX_NEWS)).map(add_prefix)

sample_cnn

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 10
})

The dataset contains three columns: article, highlights, and id. To use LangSmith, we need to create a dataset in LangSmith format.

LangSmith expects a prompt and a result. To achieve this, we will transform the article into a prompt by adding the prefix: "Summarize this news." As a result, we will use the content of highlights, which represents the summaries created by humans.

In [12]:
print(sample_cnn[0])

{'article': 'Summarize this news:\n(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV\'s "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he\'d been a busy actor for decades in theater and in Hollywood, Best didn\'t become famous until 1979, when "The Dukes of Hazzard\'s" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best\'s Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle

Now We have the Dataset with the prompt and the Reference Summary, it is time to create a Dataset in LangSmith with this information.
### Create the Dataset in Langsmith

The dataset in LangSmith is composed of an input, which is the prompt passed to the model for evaluation, and an output, which should contain what we expect the model to return.

In [13]:
import datetime

In [14]:
import uuid
input_key=['article']
output_key=['highlights']

NAME_DATASET=f"Summarize_dataset_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

In [15]:
#This creates the dataset in LangSmith with the content in sample_cnn - If you run this more than once you will get POST errors
dataset = client.upload_dataframe(
    df=sample_cnn,
    input_keys=input_key,
    output_keys=output_key,
    name=NAME_DATASET,
    description="Test Embedding distance between model summarizations",
    data_type="kv"
)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In this image, we can see an example from the dataset once it's been registered in LangSmith.

In the Input column, there is the prompt to be sent, while in the Output column, the expected output is stored.

When performing the comparison, the model will be given the prompt, and the Cosine distance between its response and the one stored in the sample dataset will be calculated.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Dataset.jpg?raw=true">

### Recovering Models From Hugging Face
Let's retrieve both models from HuggingFace. A base T5 model and a model that has been fine-tuned using the training portion of this same dataset to generate summaries.

In [16]:
from langchain import HuggingFaceHub

In [17]:
summarizer_base = HuggingFaceHub(
    repo_id="t5-base",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HF_TOKEN
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [18]:
summarizer_finetuned = HuggingFaceHub(
    repo_id="flax-community/t5-base-cnn-dm",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HF_TOKEN
)

## Defining Evaluator
The first step is to define an evaluator, where we specify the variables we want to evaluate. In our case, I have chosen to measure only the "embedding_distance."

I've left the "string_distance" as a comment in case you want to conduct a test with two evaluations instead of one.


In [19]:
from langchain.smith import run_on_dataset, RunEvalConfig
!pip install -q rapidfuzz==3.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.4 MB/s eta 0:00:00


In [20]:
#We are using just one of the multiple evaluator avaiable on LangSmith.

evaluation_config = RunEvalConfig(
    evaluators=[
        "embedding_distance",
        #"string_distance"
    ],
)



### Running Evaluator
With the same configuration, we can launch two evaluations on the same dataset. One for each of the chosen models.

In [21]:
project_name = f"T5-BASE {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

base_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_base,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-BASE 2024-07-12 10:23:50' at:
https://smith.langchain.com/o/e1be77c6-232c-5aa5-8daf-853f4875867a/datasets/9ff29ca8-6a83-4a63-8166-f8738f3fac48/compare?selectedSessions=0b4a4bdb-d6b1-49f3-8a95-3b2f5d376bbc

View all tests for Dataset Summarize_dataset_2024-07-12 10:22:24 at:
https://smith.langchain.com/o/e1be77c6-232c-5aa5-8daf-853f4875867a/datasets/9ff29ca8-6a83-4a63-8166-f8738f3fac48


Error Type: BadRequestError, Message:  (Request ID: CmrJIfnG_TuXZX-S5Pfiw)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: 7iG5cDWTK3lTSIi33gNMh)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: jF07XSW97Hl4TL4-m4Rbn)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: ByJHU5zQ1rQ_v9t9i9SNO)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: YLjoUvFxDrKcznWOJp8SY)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: 1_3Q_Sy1teh614raLKytq)

Bad request:
Authorization header is correct, but the token seems invalid


[----------------------------->                    ] 6/10

Error Type: BadRequestError, Message:  (Request ID: 0yey2JNx0myBZqR5KuwgU)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: hvSvXwACFnQV8aeGT4npZ)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: xvOIdmGJdwnYlRnc1QDdE)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: 3do2ZZH57rX_BdfH-aG4I)

Bad request:
Authorization header is correct, but the token seems invalid


[------------------------------------------------->] 10/10

In [22]:
#Ignore the error shown below
project_name = f"T5-FineTuned {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_finetuned,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-FineTuned 2024-07-12 10:24:30' at:
https://smith.langchain.com/o/e1be77c6-232c-5aa5-8daf-853f4875867a/datasets/9ff29ca8-6a83-4a63-8166-f8738f3fac48/compare?selectedSessions=0311eb85-c7bf-460a-8d65-8cf6f5092cc9

View all tests for Dataset Summarize_dataset_2024-07-12 10:22:24 at:
https://smith.langchain.com/o/e1be77c6-232c-5aa5-8daf-853f4875867a/datasets/9ff29ca8-6a83-4a63-8166-f8738f3fac48
[>                                                 ] 0/10

Error Type: BadRequestError, Message:  (Request ID: E3Fw0muUQfeQ47d21SpwM)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: ivMSWTgHTKC0mThTEHn13)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: 0U2TDHqDAV0loZJRDjn9n)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: 3A0k3qhuFKK0ASNYTo5k6)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: 6LYveODJmT0geB_dnB2bM)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: 8Ozs_NqZVl9G9VDjbyJ2T)

Bad request:
Authorization header is correct, but the token seems invalid
Error Type: BadRequestError, Message:  (Request ID: hlUFTpnq_GdEfEXRO6xOu)

Bad request:
Authorizati

[------------------------------------------------->] 10/10

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Tests.jpg?raw=true">

In the image below you can see the comparision between two tests.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareTestst.jpg?raw=true">

Well, since it has been so straightforward, why don't we try to make the comparison with an OpenAI model?

In [23]:
from langchain_openai import OpenAI
open_aillm=OpenAI(temperature=0.0)

In [24]:
project_name = f"OpenAI {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=open_aillm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'OpenAI 2024-07-12 10:24:49' at:
https://smith.langchain.com/o/e1be77c6-232c-5aa5-8daf-853f4875867a/datasets/9ff29ca8-6a83-4a63-8166-f8738f3fac48/compare?selectedSessions=7ec2669a-fbda-49f3-a3c3-28e5806d4c84

View all tests for Dataset Summarize_dataset_2024-07-12 10:22:24 at:
https://smith.langchain.com/o/e1be77c6-232c-5aa5-8daf-853f4875867a/datasets/9ff29ca8-6a83-4a63-8166-f8738f3fac48
[------------------------------------------------->] 10/10

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareOpenAI_HF.jpg?raw=true">

The experiment with the OpenAI model has yielded the best results. But, be aware! As we can see, there is a cost involved since we are using an API, and it needs to be paid for.

Another crucial piece of information is that we can view performance data for the models. This data could also be useful for minimally evaluating our inference server.